In [ ]:
import os
import re
import sys
import json
import requests
import datetime
import numpy as np
import pandas as pd

In [ ]:
# targets for demo (except for CLTA - see below)
target_names = [
    'ATL2',
    'ATL3',
    'LAMP1',
    'LMAN1',
    'MTOR',
    'OSBPL8',
    'RAB14',
    'SEC13',
    'SEC24D',
    'SPTLC1',
    'TAF12',
    'VAPA',
    'ATP2B1',
    'LMNA',
    'POLR1D',
    'LMNB1',
    'POLR1A',
    'POLR1C',
    'CSNK2A2',
    'CSNK2A1',
    'TOP2A'
]

len(target_names)

In [ ]:
# download polyclonalline data from the pipeline database API
# (this include metadata, FACS data, and some sequencing results)
result = requests.get('http://localhost:5000/polyclonallines')
all_data = result.json()

In [ ]:
# select the data for the demo targets
data = [row for row in all_data if row['target_name'] in target_names]

In [ ]:
# CLTA is a control; pick the one from plate1
data.append(
    [row for row in all_data if row['target_name']=='CLTA' and row['plate_design_id']=='P0001'][0])

In [ ]:
len([row['target_name'] for row in data])

In [ ]:
# key the data by target_name
ddata = {}
for row in data:
    ddata[row['target_name']] = row

In [ ]:
with open('../src/demo/data/20190816_pipeline-metadata.json', 'w') as file:
    json.dump(ddata, file)

In [ ]:
# generate the data for the tpm-vs-GFP scatterplot 
expression_data = []
for row in all_data:
    expression_data.append({
        'target_name': row['target_name'],
        'tpm': np.log10(row['hek_tpm']) if row.get('hek_tpm') else None,
        'gfp': row['facs_results'].get('rel_median_log')
    })

In [ ]:
with open('../src/demo/data/20190819_expression-data.json', 'w') as file:
    json.dump(expression_data, file)